In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [11]:
ds = pd.read_csv('./titanic.csv')#train data
dtest =pd.read_csv('./test.csv')#test data
print dtest.columns
dtest.head(n=20)

Index([u'PassengerId', u'Pclass', u'Name', u'Sex', u'Age', u'SibSp', u'Parch',
       u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [16]:
#mean age calculated

mean_Age = ds['Age'].mean()
#blank columns=mean age
ds['Age'] = ds['Age'].fillna(int(mean_Age))
dtest['Age'] = dtest['Age'].fillna((int(mean_Age)))    
sex_mapping = {
    'male': 0,
    'female': 1
}
sex = ds['Sex'].values             
sex_test=dtest['Sex'].values
print sex.shape,sex_test.shape
quant_sex = np.zeros(sex.shape)             #intialise two zero tuples for sex
quant_sex2 = np.zeros(sex_test.shape)
for ix in range(quant_sex.shape[0]):
    quant_sex[ix] = sex_mapping[sex[ix]]          #loop to map male,female key value pairs to sex_mapping dictionary
    
for ix in range(quant_sex2.shape[0]):
    quant_sex2[ix] = sex_mapping[sex_test[ix]]    

(891,) (418,)


In [18]:
data = np.asarray([ds['Age'], ds['Pclass'], ds['SibSp'],
        ds['Parch'], ds['Fare'], quant_sex,
        ds['Survived']]).astype("float").T                     #make new data contaioning only relevant data columns
Y = ds['Survived'].values
cols = ['Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Sex', 'Survived']
data_test = np.asarray([dtest['Age'], dtest['Pclass'], dtest['SibSp'],
        dtest['Parch'], dtest['Fare'], quant_sex2]).astype("float").T
print data_test[0]
# ds.head(n=1)

[ 34.5      3.       0.       0.       7.8292   0.    ]


In [19]:
def data_split(dataset, col_id, value):
    data_right = []
    data_left = []
    
    # send greater values to right and smaller or equal to left
    for ix in range(dataset.shape[0]):
        if dataset[ix, col_id] <= value:
            data_left.append(dataset[ix, :])
        else:
            data_right.append(dataset[ix, :])
    return np.asarray(data_right), np.asarray(data_left)

In [24]:
x, y = data_split(data[:100], 1, 1)
print x.shape
print y.shape

print cols
print x[:, -1].mean()
print y[:, -1].mean()

print '-----------------'

p1 = entropy(x, -1)
i1 = information_gain(p1)

p2 = entropy(y, -1)
i2 = information_gain(p2)

print i1, i2
print i1 + i2

(79, 7)
(21, 7)
['Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Sex', 'Survived']
0.392405063291
0.47619047619
-----------------


TypeError: information_gain() takes exactly 3 arguments (1 given)

In [25]:
def entropy(dataset, col=-1):
    if dataset.shape[0] == 0:
        return 0
    p = dataset[:, col].mean()
    if p == 1 or p == 0:
        return 0.0
    ent = (-1.0 * p * np.log2(p)) + (-1.0 * (1.0 - p) * np.log2(1.0 - p))
    return ent
    
def information_gain(parent_set, child_1, child_2):
    # print parent_set.shape, child_1.shape, child_2.shape
    ent_p = entropy(parent_set)
    ent_c1 = entropy(child_1)
    ent_c2 = entropy(child_2)
    
    f = float(child_1.shape[0])/parent_set.shape[0]
    Ig = ent_p - (f*ent_c1 + (1-f)*ent_c2)
    return Ig

In [54]:
INF = 100000
class DT:
    def __init__(self, depth=0, max_depth=10):
        self.left = None
        self.right = None
        self.col_id = None
        self.value = None
        self.depth = depth
        self.max_depth = max_depth
        self.n_survival = None
        self.n_death = None
    
    def get_best_gain(self, dataset):
        
        n_cols = 6 # number of columns in data
        all_gains = []
        
        self.n_survival = dataset[:, -1].sum()
        self.n_death = dataset.shape[0] - self.n_survival
        
        for cx in range(n_cols):
            mean_val_cx = dataset[:, cx].mean()
            right, left = data_split(dataset, cxu, mean_val_cx)    #automatic assigning of two returning values toright,left in order
            
            total_gain = information_gain(dataset, right, left)
            all_gains.append(total_gain)
            
        self.col_id = np.asarray(all_gains).argmax()
        self.value = dataset[:, self.col_id].mean()
        
        data_right, data_left = data_split(dataset, self.col_id, self.value)
        
        if data_left.shape[0] > 0 and self.depth < self.max_depth:
            self.left = DT(depth=self.depth+1, max_depth=self.max_depth)
            self.left.get_best_gain(data_left)
            
        if data_right.shape[0] > 0 and self.depth < self.max_depth:
            self.right = DT(depth=self.depth+1, max_depth=self.max_depth)
            self.right.get_best_gain(data_right)
        return
    
    def predict(self, example):
        exp_val = example[self.col_id]
        # print self.n_survival, self.n_death, self.depth
        # print self.col_id, exp_val, self.value, '-------------'
        if exp_val <= self.value:
            if not self.left == None:
                # print 'going left'
                return self.left.predict(example)
            else:
                # print 'end left'
                return self.decide()
        else:
            if not self.right == None:
                # print 'going right'
                return self.right.predict(example)
            else:
                # print 'end right'
                return self.decide()
    
    def decide(self):
        # print self.depth
        if self.n_survival >= self.n_death:
            return 1
        else:
            return 0


In [55]:
# dt = DT(max_depth=10)
# dt.get_best_gain(data)

In [58]:
split = int(0.80 * data.shape[0])
X_train = data[:, :-1]
y_train = data[:, -1]
y_tr_pred = np.zeros(y_train.shape)

X_test = data_test[:, :]
y_test = data_test[:, -1]

print X_train.shape, X_test.shape, y_test.shape

y_pred = np.zeros((y_test.shape[0],2))
count=892
print y_pred.shape
"""all_depths = range(2, 16)
acc_train = []
acc_test = []

for dpx in all_depths:"""
dt = DT(max_depth=3)
dt.get_best_gain(X_train)
"""
    correct = 0
    cr_train = 0

    for yx in range(y_train.shape[0]):
        y_tr_pred[yx] = dt.predict(X_train[yx, :-1])
        if y_tr_pred[yx] == y_train[yx]:
            cr_train += 1
"""
for yx in range(y_pred.shape[0]):
    y_pred[yx][0] = count
    y_pred[yx][1] = dt.predict(X_test[yx, :])
    count=count+1

print y_pred.shape
df=pd.DataFrame(data=y_pred,columns=["PassengerId","Survived"])
df.to_csv("ans.csv",sep='\t',encoding='utf-8')

(891, 7) (418, 6) (418,)
(418, 2)
(418, 2)


In [42]:
plt.figure(0)
plt.plot(all_depths, acc_train, 'g')
plt.plot(all_depths, acc_test, 'r')

plt.show()

NameError: name 'all_depths' is not defined

In [47]:

print y_test[:10]

[[ 892.    0.]
 [ 893.    0.]
 [ 894.    1.]
 [ 895.    1.]
 [ 896.    0.]
 [ 897.    0.]
 [ 898.    0.]
 [ 899.    1.]
 [ 900.    1.]
 [ 901.    0.]]
[ 0.  1.  0.  0.  1.  0.  1.  0.  1.  0.]


In [41]:
n = 300
ex = data[n, :-1]
print ex, data[n, -1]
print cols
dt.predict(ex)

[ 29.     3.     0.     0.     7.75   1.  ] 1.0
['Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Sex', 'Survived']


0